In [ ]:
%%classpath add mvn
org.clustering4ever clustering4ever_2.11 0.9.6

In [ ]:
%%classpath add mvn
org.apache.spark spark-core_2.11 2.4.3

# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

# C4E Scala Epsilon Proximity

In [ ]:
import org.clustering4ever.clustering.epsilonproximity.scala.EpsilonProximityScalar
import org.clustering4ever.math.distances.scalar.Euclidean
import org.clustering4ever.clusterizables.EasyClusterizable
import org.clustering4ever.vectors.ScalarVector
import org.clustering4ever.clustering.indices.MultiExternalIndicesLocal
import scala.io.Source
import scala.collection.mutable
import scala.collection.parallel

In [ ]:
%%bash
wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/labels

### Load data

In [ ]:
val path = "/tmp/aggregation.csv"
val data = scala.io.Source.fromFile(path).getLines.toSeq.par
  .map( x => x.split(",").map(_.toDouble)).zipWithIndex
  .map{ case (v, id) => EasyClusterizable(id.toLong, ScalarVector(v)) }
val labelsPath = "/tmp/labels"
val labels = scala.io.Source.fromFile(labelsPath).getLines.toSeq.map(_.toInt)

## Parameters 

In [ ]:
type SeqType = mutable.ArrayBuffer[Double]
val metric = Euclidean(false)
val eps = "knn:25"
val eps2 = "eps:1.5"

## Run Epsilon Proximity 

In [ ]:
val t1 = System.nanoTime
val modelEps = EpsilonProximityScalar(eps, metric).fit(data)
val t2 = System.nanoTime
(t2 - t1) / 1000000000D

In [ ]:
val clusterizedd = modelEps.obtainInputDataClustering(data)
val rawd = clusterizedd.map(_.v.vector.toArray).toArray
val labelsPred = clusterizedd.map(_.clusterIDs.last).toArray

## Compute NMI with founded class

In [ ]:
val indices = MultiExternalIndicesLocal(labelsPred zip labels)
val nmi = indices.nmiSQRT

In [ ]:
val arand = indices.arand